In [1]:
import ShapeRetrieval: ShapeRetrieval as SR
using ShapeRetrieval: LearnedTimeDiffusionBlock
import ShapeRetrieval: Mesh, heat_integrator
using LinearAlgebra
using SparseArrays
using Flux
using Zygote

### Visualization Tools

In [2]:
bunny = SR.load_obj("./meshes/gourd.obj")
bunny = SR.normalize_mesh(bunny)
println("nv: $(bunny.nv) nf: $(bunny.nf) Area: ", sum(bunny.vertex_area))
println(count(x->x != 0, bunny.cot_laplacian) / (bunny.nv ^2 )) 

nv: 326 nf: 648 Area: 2.681867885523121
0.02135947909217509

In [3]:
function spectral_loss(model, x, λ, ϕ, A, y) 
    norm(model(x, λ, ϕ, A) - y)
end

spectral_loss (generic function with 1 method)

## Example Prediction using Spectral Mode

In [ ]:
m = LearnedTimeDiffusionBlock(2, :implicit)
heat_signal = zeros(bunny.nv,2)
heat_signal[1, 1] = 1.0
heat_signal[200,2] = 1.0
# λ, ϕ, A = SR.get_diffusion_inputs(bunny, :spectral)
L, A = SR.get_diffusion_inputs(bunny, :implicit)
m(heat_signal, L, M, A)

In [12]:
println(sum(bunny.face_area))
rescaled_bunny = SR.normalized_area_mesh(bunny)
println(sum(rescaled_bunny.face_area))

2.681867885523122


UndefVarError: UndefVarError: `normalized_area_mesh` not defined

In [19]:
v_sphere, f_sphere = SR.get_in_sphere(bunny, 3, 0.1)
v_reach, f_reach = SR.connected_mesh_in_sphere(bunny, 3, 0.1)
.
# should be false. check that connection doesn't add any additional verts/faces
println(any(.!v_sphere .& v_reach))
println(any(.!f_sphere .& f_reach))

false
false


In [26]:
centered = SR.rotate_mesh_random(bunny)

Mesh([0.6680656438139231 0.6456967881478018 … 0.7598759276010049 0.7507491439186327; 0.14813231996937676 0.15045699156185097 … 0.4595666882415629 0.46665654211297; 0.05194462291877181 0.027852161954583926 … -0.22134146991517634 -0.2676357379531484], [1 4 … 325 324; 2 5 … 324 325; 3 3 … 319 326], [0.5416442809283524 0.4851086484979241 … -0.7899519585213421 -0.5602970596675746; -0.6241175671330316 -0.47186169888749907 … 0.33767207252969544 0.7950717535177043; -0.5631152060933483 -0.7362174517596826 … 0.5118139062801983 0.23222427023494624], [0.5416442809283524 0.4851086484979241 … -0.7899519585213421 -0.5602970596675746; -0.6241175671330316 -0.47186169888749907 … 0.33767207252969544 0.7950717535177043; -0.5631152060933483 -0.7362174517596826 … 0.5118139062801983 0.23222427023494624], [2.9064900846146684 2.3002803487647014 … -0.39181274370118635 0.2594978305544219; -3.454141455823979 -2.780267544150153 … 2.959638811066684 0.7860627791291807; -2.02617960780711 -3.3104689254261115 … 1.27654

In [31]:
SR.cache_operators_for_folder("meshes")

meshes/bunny.obj


meshes/cat.obj


InterruptException: InterruptException:

In [28]:
expected = SR.save_operators_to_cache(bunny, "bunny")
actual = SR.load_cached_operators("bunny")
for (a, b) in zip(expected, actual)
    println()
    println(typeof(a), "\n", typeof(b))
    println(norm(a-b))
end


SparseMatrixCSC{Float64, Int64}
SparseMatrixCSC{Float64, Int64}
0.0

Vector{Float32}
Vector{Float32}
0.0

Vector{Float32}
Vector{Float32}
0.0

Matrix{Float32}
Matrix{Float32}
0.0

SparseMatrixCSC{Float32, Int64}
SparseMatrixCSC{Float32, Int64}
0.0

SparseMatrixCSC{Float32, Int64}
SparseMatrixCSC{Float32, Int64}
0.0


#### Sample Training 

In [17]:
λ, ϕ, A = SR.get_diffusion_inputs(bunny, :implicit)

opt_state = Flux.setup(Adam(), m);

y_true = m(heat_signal, λ, ϕ, A)
println("t_init ", m.diffusion_time)
m = LearnedTimeDiffusionBlock(2, :implicit)
println("t_start: ", m.diffusion_time)
println("start cost ", spectral_loss(m, heat_signal, λ, ϕ, A, y_true))
@time for i=1:2000
    x, y  = heat_signal, y_true
    grad = gradient(spectral_loss, m, x, λ, ϕ, A, y)
    Flux.update!(opt_state, m, grad[1])
end
println(spectral_loss(m,heat_signal, λ, ϕ, A, y_true))
println(m)

MethodError: MethodError: no method matching get_diffusion_inputs(::Mesh, ::Symbol)

Closest candidates are:
  get_diffusion_inputs(::Mesh, !Matched::Type{<:ShapeRetrieval.DiffusionMode})
   @ ShapeRetrieval ~/Documents/MIT/_S2023/ShapeRetrieval/src/utils.jl:31


In [6]:
m(heat_signal, SR.get_diffusion_inputs(bunny)...)
println(m.diffusion_time)

(200, 2)


[2.9274950637401447, 0.7993044171182087]


### AD with Implicit Mode

In [7]:
m = LearnedTimeDiffusionBlock(2, :implicit)
Flux.trainable(m::LearnedTimeDiffusionBlock) = (m.diffusion_time,)

function implicit_loss(model, x, L, M, A::Vector{Float64}, y)
    norm(model(x, L, M, A) - y)
end

inputs = SR.get_diffusion_inputs(bunny,:implicit)
m(heat_signal, inputs...)

0.30148325561767775

In [49]:
heat_signal = zeros(bunny.nv)
heat_signal[[1, 300]] .= 1.0
L, M, A = SR.get_diffusion_inputs(bunny, :implicit)
display(m(heat_signal, L, M, A))
data = [(heat_signal, bunny, 0.0),]

opt_state = Flux.setup(Adam(), m)

for i=1:1000
    for d in data
        x, mesh, y  = d
        grad = gradient(implicit_loss, m, x, L,M,A, y)
        Flux.update!(opt_state, m, grad[1])
    end
end

display(m(heat_signal, L, M, A))

0.3014815281221147

0.301481517244275